In [96]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils import shuffle
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE, ADASYN

In [97]:
train = pd.read_csv("/kaggle/input/playground-series-s3e3/train.csv").drop("id", axis=1)
train.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,...,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,Male,...,80,1,10,2,3,10,0,7,8,0
1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,Male,...,80,1,4,3,3,4,2,0,3,0
2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,Male,...,80,2,4,3,3,3,2,1,2,0
3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,Female,...,80,0,15,1,1,6,0,0,2,0
4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,Female,...,80,0,31,0,3,31,14,4,10,1


In [98]:
"""from pandas_profiling import ProfileReport
profile = ProfileReport(train)
profile.to_file(output_file="EmployeeAttrition.html")"""

'from pandas_profiling import ProfileReport\nprofile = ProfileReport(train)\nprofile.to_file(output_file="EmployeeAttrition.html")'

In [99]:
original = pd.read_csv("/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv").drop("EmployeeNumber", axis=1)
original["Attrition"] = (original["Attrition"] == 'Yes').astype(int)
original = original[train.columns]

In [100]:
#train = train.append(original[original[["Attrition"]].all(1)])
train = train.append(original)
test = pd.read_csv("/kaggle/input/playground-series-s3e3/test.csv")
y = train.Attrition
test_ids = test["id"]
test.drop("id", inplace=True, axis=1)
train.drop("Attrition", axis=1, inplace=True)
df = train.append(test)

In [101]:
df.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,36,Travel_Frequently,599,Research & Development,24,3,Medical,1,4,Male,...,2,80,1,10,2,3,10,0,7,8
1,35,Travel_Rarely,921,Sales,8,3,Other,1,1,Male,...,4,80,1,4,3,3,4,2,0,3
2,32,Travel_Rarely,718,Sales,26,3,Marketing,1,3,Male,...,4,80,2,4,3,3,3,2,1,2
3,38,Travel_Rarely,1488,Research & Development,2,3,Medical,1,3,Female,...,3,80,0,15,1,1,6,0,0,2
4,50,Travel_Rarely,1017,Research & Development,5,4,Medical,1,2,Female,...,3,80,0,31,0,3,31,14,4,10


In [102]:
to_drop = [col for col in df.columns if df[col].nunique()==1]
df.drop(to_drop, axis=1, inplace=True)

In [103]:
cols = [col for col in df.columns if df[col].nunique()<20]
for c in cols:
    print(c, " : ", df[c].unique())

BusinessTravel  :  ['Travel_Frequently' 'Travel_Rarely' 'Non-Travel']
Department  :  ['Research & Development' 'Sales' 'Human Resources']
Education  :  [ 3  4  1  2  5 15]
EducationField  :  ['Medical' 'Other' 'Marketing' 'Life Sciences' 'Technical Degree'
 'Human Resources']
EnvironmentSatisfaction  :  [4 1 3 2 0]
Gender  :  ['Male' 'Female']
JobInvolvement  :  [3 1 2 4 0]
JobLevel  :  [1 2 5 3 4 7]
JobRole  :  ['Laboratory Technician' 'Sales Representative' 'Sales Executive'
 'Healthcare Representative' 'Manager' 'Manufacturing Director'
 'Research Scientist' 'Human Resources' 'Research Director']
JobSatisfaction  :  [4 1 2 3]
MaritalStatus  :  ['Married' 'Divorced' 'Single']
NumCompaniesWorked  :  [ 1  0  3  5  2  8  6  7  9  4 17]
OverTime  :  ['Yes' 'No']
PercentSalaryHike  :  [13 17 14 11 12 15 21 16 18 19 24 22 23 20 25]
PerformanceRating  :  [3 4]
RelationshipSatisfaction  :  [2 4 3 1]
StockOptionLevel  :  [1 2 0 3 4]
TrainingTimesLastYear  :  [2 3 1 0 5 4 6]
WorkLifeBalance  :

In [104]:
cat_col = [col for col in df.columns if df[col].nunique()<20]
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
df[cat_col] = encoder.fit_transform(df[cat_col])

In [105]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = scaler.fit_transform(df)

In [106]:
X_train = df[:-len(test), :]
X_test = df[-len(test): , :]

In [107]:
"""X_train, y = ADASYN(random_state=42).fit_resample(X_train, y)
X_train, y = shuffle(X_train, y)"""

'X_train, y = ADASYN(random_state=42).fit_resample(X_train, y)\nX_train, y = shuffle(X_train, y)'

In [108]:
def cross_validate(model, X, y):
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    all_scores = []
    for fold_id, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        model.fit(X_tr, y_tr)
        
        y_pred = model.predict_proba(X_val)[:, 1]
        
        auc = roc_auc_score(y_val, y_pred)
        
        all_scores.append(auc)
    
    avg_auc = np.mean(all_scores)
    
    print(f"Avg AUC: {avg_auc}")
    return avg_auc

In [109]:
print('The ratio of majority class to minority class is {}:1.' \
      .format(int(y[y == 0].size/y[y == 1].size)))

The ratio of majority class to minority class is 6:1.


In [110]:
def tryModel(modelList, X_train, y_train):
    for model in modelList:
        print(model)
        cross_validate(model, X_train, y_train)

In [111]:
catboost_params = {'loss_function': 'CrossEntropy',
                     'learning_rate': 0.76,
                     'l2_leaf_reg': 0.014,
                     'colsample_bylevel': 0.06,
                     'depth': 1,
                     'boosting_type': 'Plain',
                     'bootstrap_type': 'Bernoulli',
                     'min_data_in_leaf': 18,
                     'one_hot_max_size': 14,
                     'subsample': 0.99}
lgbm_params = {'n_estimators': 407,
                 'learning_rate': 0.1,
                 'num_leaves': 195,
                 'max_depth': 9,
                 'min_data_in_leaf': 46,
                 'lambda_l1': 0.01,
                 'lambda_l2': 0.6,
                 'min_gain_to_split': 1.42,
                 'bagging_fraction': 0.45,
                 'feature_fraction': 0.3}
xgb_params = {'n_estimators': 150,
                 'max_depth': 3,
                 'learning_rate': 0.1,
                 'min_child_weight': 4,
                 'subsample': 0.7,
                 'colsample_bytree': 0.3,
             }
gbc_params = {'learning_rate': 0.1,
              'max_depth': 1,
              'n_estimators': 500}
xgb_params1 = {'max_depth': 1,
               'learning_rate': 0.2108601940138979,
               'n_estimators': 218,
               'min_child_weight': 8,
               'gamma': 0.22409670453974548,
               'subsample': 0.6498508356594279,
               'colsample_bytree': 0.9793380319815592,
               'reg_alpha': 0.49112506147518226,
               'reg_lambda': 0.058351357422039285}
xgb_params2 = {'max_depth': 1,
              'learning_rate': 0.38159544738365864,
              'n_estimators': 212,
              'min_child_weight': 2,
              'gamma': 0.7693827218695433,
              'subsample': 0.7187150336992639,
              'colsample_bytree': 0.3262539070555781,
              'reg_alpha': 0.6436046033432924,
              'reg_lambda': 0.882904959066567}

In [112]:
models = [
    #RandomForestClassifier(),
    CatBoostClassifier(verbose=False, **catboost_params), #0.9309
    XGBClassifier(**xgb_params), #0.94102
    XGBClassifier(**xgb_params1), #0.93168
    XGBClassifier(**xgb_params2), #0.93246
    LGBMClassifier(**lgbm_params), #0.93012
    #SVC(probability=True),
    GradientBoostingClassifier(), #0.93308
    GradientBoostingClassifier(**gbc_params),#0.93962
    #HistGradientBoostingClassifier()
]
#xgb + gbc 0.94148
#xgb + lgb + cat 0.9393
#xgb + cat + gbc 0.94024
#xgb + cat + gbc + lgb 0.93728
#gbc (tuned) 0.93962
#xgb + gbc (tuned) 0.94273
# xgb (param1) + gbc(tuned) 0.93884
#xgb(param2) + gbc(tuned) 0.93759
#xgb(param1) 0.93168
#xgb(param2) 0.93246

In [113]:
tryModel(models,X_train, np.array(y))

Avg AUC: 0.8468667842069542
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=4, missing=nan,
              monotone_constraints=None, n_estimators=150, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)
Avg AUC: 0.8526709860121857
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9793380319815592, early_stopping_rounds=None,
          

In [114]:
"""gbc = GradientBoostingClassifier()
parameters = {
    "n_estimators":[5,50,250,500],
    "max_depth":[1,3,5,7,9],
    "learning_rate":[0.01,0.1,1,10,100]
}
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(gbc,parameters,cv=5, scoring='roc_auc')
cv.fit(X_train, y)
print(cv.best_params_)"""

'gbc = GradientBoostingClassifier()\nparameters = {\n    "n_estimators":[5,50,250,500],\n    "max_depth":[1,3,5,7,9],\n    "learning_rate":[0.01,0.1,1,10,100]\n}\nfrom sklearn.model_selection import GridSearchCV\ncv = GridSearchCV(gbc,parameters,cv=5, scoring=\'roc_auc\')\ncv.fit(X_train, y)\nprint(cv.best_params_)'

In [115]:
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_float('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'auc',
        'use_label_encoder': False
    }

    # Model
    optuna_model = XGBClassifier(**params)
    scores = cross_val_score(optuna_model, X_train, y, cv = 5, scoring='roc_auc')
    return scores.mean()

In [116]:
"""import optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print(study.best_trial.params)"""

"import optuna\nstudy = optuna.create_study(direction='maximize')\nstudy.optimize(objective, n_trials=100)\nprint(study.best_trial.params)"

In [117]:
y_probs = []
for model in models:
    y_prob = model.predict_proba(X_test)[:, 1]
    y_probs.append(y_prob)

In [118]:
final_preds = np.column_stack(y_probs).mean(axis=1)
submission = pd.DataFrame({"id": test_ids, "Attrition": final_preds})
submission.to_csv("submission.csv", index=False)